In [3]:
import pandas as pd
from itertools import combinations

# Load the input file
data_file_path = 'C:/Users/saiki/OneDrive/Desktop/Sampledata.csv'
data = pd.read_csv(data_file_path, encoding='ISO-8859-1')

# Define the output file path
output_file_path = 'C:/Users/saiki/OneDrive/Desktop/output1.txt'
with open(output_file_path, 'w') as output_file:

    # Prompt for primary keys
    primary_key_columns = input("Please list the primary key column(s), separated by commas: ").split(',')
    primary_key_columns = [col.strip() for col in primary_key_columns]

    # Prompt for functional dependencies
    func_deps = {}
    print("Enter Functional Dependencies in 'X,Y -> Z,W' format (type 'end' when done):")
    while True:
        fd_input = input()
        if fd_input.lower() == 'end':
            break
        try:
            lhs, rhs = fd_input.split("->")
            lhs_attrs = [attr.strip() for attr in lhs.split(',')]
            rhs_attrs = [attr.strip() for attr in rhs.split(',')]
            func_deps[tuple(lhs_attrs)] = rhs_attrs
            output_file.write(f"Functional Dependency: {lhs} -> {rhs}\n")
        except ValueError:
            print("Incorrect format. Use 'X,Y -> Z,W'.")

    # Get the target normal form level
    target_form = input("Enter the desired highest normal form (1: 1NF, 2: 2NF, 3: 3NF, B: BCNF, 4: 4NF, 5: 5NF): ")
    if target_form in ["1", "2", "3", "4", "5"]:
        target_form = int(target_form)

    # Prompt for multi-valued dependencies if target form is 4NF or higher
    mvd_dependencies = {}
    if target_form >= 4:
        print("Specify Multi-Valued Dependencies in 'A ->> B' format (type 'end' when done):")
        while True:
            mvd_input = input()
            if mvd_input.lower() == 'end':
                break
            try:
                lhs, rhs = mvd_input.split("->>")
                lhs, rhs = lhs.strip(), rhs.strip()
                if lhs in mvd_dependencies:
                    mvd_dependencies[lhs].append(rhs)
                else:
                    mvd_dependencies[lhs] = [rhs]
                output_file.write(f"Multi-Valued Dependency: {lhs} ->> {rhs}\n")
            except ValueError:
                print("Incorrect format. Use 'A ->> B'.")

    # Prompt for join dependencies if target form is 5NF
    jd_dependencies = []
    if target_form == 5:
        print("Specify Join Dependencies in 'X,Y' format (type 'end' when done):")
        while True:
            jd_input = input()
            if jd_input.lower() == 'end':
                break
            jd_attrs = [attr.strip() for attr in jd_input.split(',')]
            jd_dependencies.append(jd_attrs)
            output_file.write(f"Join Dependency: {', '.join(jd_attrs)}\n")

    # Function to enforce 1NF by handling non-atomic attributes
    def enforce_1nf(df):
        atomic_cols = []
        for col in df.columns:
            if df[col].apply(lambda x: isinstance(x, list)).any():
                atomic_cols.append(col)
                df = df.explode(col)
        return df, atomic_cols

    # Function to check 2NF
    def check_2nf(df, pk_columns, deps):
        non_pk_columns = [col for col in df.columns if col not in pk_columns]
        for lhs, rhs in deps.items():
            if set(lhs).issubset(pk_columns) and any(attr in non_pk_columns for attr in rhs):
                return False
        return True

    # Function to check 3NF
    def check_3nf(df, pk_columns, deps):
        for lhs, rhs in deps.items():
            if set(lhs) != set(pk_columns) and not set(rhs).issubset(pk_columns):
                return False
        return True

    # Function to check BCNF
    def check_bcnf(df, pk_columns, deps):
        for lhs, rhs in deps.items():
            if not set(lhs).issubset(pk_columns):
                return False
        return True

    # Function to apply 4NF with multi-valued dependencies
    def apply_4nf(df, mv_deps):
        decomposed_tables = []
        remaining_columns = list(df.columns)

        for lhs, rhs_list in mv_deps.items():
            for rhs in rhs_list:
                decomposed_table = df[[lhs, rhs]].drop_duplicates()
                decomposed_tables.append((decomposed_table, [lhs, rhs]))
                if rhs in remaining_columns:
                    remaining_columns.remove(rhs)
                output_file.write(f"4NF Decomposition: Created table with {lhs} ->> {rhs}\n")

        main_table = df[remaining_columns].drop_duplicates()
        return [(main_table, primary_key_columns)] + decomposed_tables

    # Function to apply 5NF with join dependencies
    def apply_5nf(df, jd_deps):
        decomposed_tables = []

        for jd in jd_deps:
            if set(jd).issubset(df.columns):
                decomposed_table = df[jd].drop_duplicates()
                decomposed_tables.append((decomposed_table, jd))
                output_file.write(f"5NF Decomposition: Created table for JD ({', '.join(jd)})\n")

        if not decomposed_tables:
            output_file.write("No further 5NF decomposition required.\n")
        return [(df, primary_key_columns)] + decomposed_tables

    # SQL Query Generator and Schema Representation
    def generate_sql_schema(table_name, df, pk_columns):
        sql_query = f"CREATE TABLE {table_name} (\n"
        schema_info = f"Table: {table_name}\nAttributes:\n"

        for col, dtype in zip(df.columns, df.dtypes):
            col_type = 'INT' if 'int' in str(dtype) else 'VARCHAR(255)'
            sql_query += f"  {col} {col_type},\n"
            schema_info += f"  - {col} ({col_type})\n"

        pk_str = ", ".join(pk_columns)
        sql_query += f"  PRIMARY KEY ({pk_str})\n);"
        schema_info += f"Primary Key(s): {pk_str}\n\n"

        print(sql_query)
        output_file.write(sql_query + "\n\n" + schema_info)

    # Main Normalization Process
    def normalize_data(df, pk_columns, f_deps, mv_deps, target_form, jd_deps=[]):
        df, atomic_cols = enforce_1nf(df)
        output_file.write("Applied 1NF\n" if not atomic_cols else f"1NF: Converted non-atomic columns {atomic_cols}\n")
        generate_sql_schema('Table_1NF', df, pk_columns)

        current_nf = 1
        if target_form >= 2 and check_2nf(df, pk_columns, f_deps):
            output_file.write("2NF validation successful.\n")
            current_nf = 2
        else:
            output_file.write("2NF decomposition needed.\n")

        if target_form >= 3 and check_3nf(df, pk_columns, f_deps):
            output_file.write("3NF validation successful.\n")
            current_nf = 3
        else:
            output_file.write("3NF decomposition needed.\n")

        if target_form >= 4 and check_bcnf(df, pk_columns, f_deps):
            output_file.write("BCNF validation successful.\n")
            current_nf = "BCNF"
        else:
            output_file.write("BCNF decomposition needed.\n")

        decomposed_tables = [(df, pk_columns)]
        if target_form >= 4:
            decomposed_tables = apply_4nf(df, mv_deps)
            for i, (table, keys) in enumerate(decomposed_tables, start=1):
                table_name = f"Table_4NF_{i}"
                generate_sql_schema(table_name, table, keys)
            current_nf = 4 if current_nf != "BCNF" else current_nf

        if target_form == 5:
            decomposed_tables = apply_5nf(df, jd_deps)
            for i, (table, keys) in enumerate(decomposed_tables, start=1):
                table_name = f"Table_5NF_{i}"
                generate_sql_schema(table_name, table, keys)

        output_file.write(f"The highest normal form achieved: {current_nf}\n")

    # Execute the normalization
    normalize_data(data, primary_key_columns, func_deps, mvd_dependencies, target_form, jd_dependencies)

print(f"Normalization results saved to {output_file_path}")


Please list the primary key column(s), separated by commas: OrderID, CustomerID, DrinkID, FoodID
Enter Functional Dependencies in 'X,Y -> Z,W' format (type 'end' when done):
OrderID -> CustomerID
end
Enter the desired highest normal form (1: 1NF, 2: 2NF, 3: 3NF, B: BCNF, 4: 4NF, 5: 5NF): 4
Specify Multi-Valued Dependencies in 'A ->> B' format (type 'end' when done):
OrderID ->> DrinkID
done
Incorrect format. Use 'A ->> B'.
end
CREATE TABLE Table_1NF (
  OrderID INT,
  Date VARCHAR(255),
  PromocodeUsed VARCHAR(255),
  TotalCost VARCHAR(255),
  TotalDrinkCost VARCHAR(255),
  TotalFoodCost VARCHAR(255),
  CustomerID INT,
  CustomerName VARCHAR(255),
  DrinkID INT,
  DrinkName VARCHAR(255),
  DrinkSize VARCHAR(255),
  DrinkQuantity INT,
  Milk VARCHAR(255),
  DrinkIngredient VARCHAR(255),
  DrinkAllergen VARCHAR(255),
  FoodID INT,
  FoodName VARCHAR(255),
  FoodQuantity INT,
  FoodIngredient VARCHAR(255),
  FoodAllergen VARCHAR(255),
  PRIMARY KEY (OrderID, CustomerID, DrinkID, FoodID)
)